In [8]:
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
"""
The system should function as a decision-support tool, enabling users to set intervals (e.g., daily, weekly) or define financial goals (e.g., savings targets) and receive relevant recommendations.
"""
def train_forcasting_model_for_savings(df, savings_goal, forecast_periods=12, interval='W'):
    prophet_data = pd.DataFrame({
        'ds': df.index,
        'y': df['Savings for Property (£)']
    }).reset_index(drop=False)

    # Train the model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False
    )
    model.fit(prophet_data)

    # Make future predictions
    future = model.make_future_dataframe(periods=forecast_periods, freq=interval)
    forecast = model.predict(future)

    # Calculate projected savings and shortfall
    last_date = prophet_data['ds'].max()
    projected_savings = forecast[forecast['ds'] > last_date]['yhat'].sum()
    shortfall = max(0, savings_goal - projected_savings)

    # Add goal analysis to forecast DataFrame
    forecast['savings_goal_line'] = savings_goal / forecast_periods
    forecast['cumulative_savings'] = forecast['yhat'].cumsum()

    # Add status indicators
    forecast['on_track'] = forecast['cumulative_savings'] >= (
        forecast.index * (savings_goal / len(forecast))
    )

    return forecast, shortfall

In [13]:
forcast_period = 4
expenses_limit = 1000
savings_goal = 10000
today_date = pd.to_datetime('today').date()
dates = pd.date_range(start=today_date, periods=forcast_period, freq='ME')

dataFrame = pd.DataFrame({
    'Monthly Income (£)': 5000,
    'Total Expenses': 987.87,
    'Savings for Property (£)': 300
}, index=dates)


In [14]:
# Run the model
forecast, shortfall = train_forcasting_model_for_savings(
    dataFrame,
    savings_goal=savings_goal,
    forecast_periods=forcast_period,
    interval='ME'
)


In [12]:
print("Savings Forecast Analysis:")
print(f"Projected Total Savings: £{forecast['yhat'].sum():,.2f}")
print(f"Shortfall from Goal: £{shortfall:,.2f}")

# Monthly breakdown
print("\nMonthly Savings Forecast:")
forecast_summary = pd.DataFrame({
    'Date': forecast['ds'],
    'Predicted Savings': forecast['yhat'].round(2),
    'Cumulative Savings': forecast['cumulative_savings'].round(2),
    'On Track': forecast['on_track']
})
print(forecast_summary)

# Goal tracking status
months_on_track = forecast['on_track'].sum()
total_months = len(forecast)
print(f"\nMonths on track to meet goal: {months_on_track} out of {total_months}")

print("\nRecommendations:")
if shortfall > 0:
    monthly_increase_needed = shortfall / forcast_period
    print(f"- Increase monthly savings by £{monthly_increase_needed:.2f} to meet your goal")
    if dataFrame['Total Expenses'].mean() > expenses_limit:
        print(f"- Reduce monthly expenses (currently exceeding £{expenses_limit:,.2f} limit)")
else:
    print("- You're on track to meet your savings goal!")

Savings Forecast Analysis:
Projected Total Savings: £2,400.00
Shortfall from Goal: £8,800.00

Monthly Savings Forecast:
        Date  Predicted Savings  Cumulative Savings  On Track
0 2024-11-30              300.0               300.0      True
1 2024-12-31              300.0               600.0     False
2 2025-01-31              300.0               900.0     False
3 2025-02-28              300.0              1200.0     False
4 2025-03-31              300.0              1500.0     False
5 2025-04-30              300.0              1800.0     False
6 2025-05-31              300.0              2100.0     False
7 2025-06-30              300.0              2400.0     False

Months on track to meet goal: 1 out of 8

Recommendations:
- Increase monthly savings by £2200.00 to meet your goal
